In [ ]:
import pandas as pd
import json

# --- Configuration ---
INPUT_FILE = 'terrorism_dataset.csv'
OUTPUT_FILE = 'globe_target.json'

TARGET_MAP = {
    'Military': 'military_police',
    'Police': 'military_police',
    'Government (General)': 'government',
    'Government (Diplomatic)': 'government',
    'Business': 'business',
    'Private Citizens & Property': 'citizens',
    'Transportation': 'transportations'
}

# --- Processing ---
try:
    # Load data (GTD standard encoding usually requires ISO-8859-1)
    df = pd.read_csv(INPUT_FILE, encoding='ISO-8859-1', low_memory=False)

    # Filter for relevant targets and map them
    mask = df['targtype1_txt'].isin(TARGET_MAP.keys())
    df_clean = df.loc[mask].copy()
    df_clean['target_slug'] = df_clean['targtype1_txt'].map(TARGET_MAP)

    # Calculate bin key (End of the 5-year block, e.g., 1970-1974 -> 1974)
    df_clean['bin_key'] = (df_clean['iyear'] // 5) * 5 + 4

    # Aggregate: Count attacks per Bin -> Country -> Target
    counts = df_clean.groupby(['bin_key', 'country_txt', 'target_slug']).size().reset_index(name='count')

    # Sort descending to bring highest counts to top
    counts.sort_values(['bin_key', 'country_txt', 'count'], ascending=[True, True, False], inplace=True)

    # Dedup to keep only the most preponderant target per country/bin
    top_targets = counts.drop_duplicates(subset=['bin_key', 'country_txt'])

    # --- JSON Construction ---
    output_data = {}
    
    # Iterate through unique bins to build the structure
    for bin_year in sorted(top_targets['bin_key'].unique()):
        bin_records = top_targets[top_targets['bin_key'] == bin_year]
        
        country_list = [
            {
                row['country_txt']: {
                    "attacks": int(row['count']),
                    "target": row['target_slug']
                }
            }
            for _, row in bin_records.iterrows()
        ]
        
        output_data[int(bin_year)] = country_list

    # --- Save Output ---
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(output_data, f, indent=4)
        
    print(f"Successfully created {OUTPUT_FILE} with {len(output_data)} time bins.")

except FileNotFoundError:
    print(f"Error: {INPUT_FILE} not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Loading and cleaning terrorism_dataset.csv...
Generating top_50_attacks_per_category.json...
